# Barnase-Barstar protein complex

In [ ]:
# Barnase-Barstar protein complex
# From Chen, R., Mintseris, J., Janin, J. and Weng, Z. (2003)
# A protein–protein docking benchmark. 
# Proteins, 52: 88-91. https://doi-org.sire.ub.edu/10.1002/prot.10390
barnase_id = "1A2P"
barnase_ch = "B"
barstar_id = "1A19"
barstar_ch = "A"
complex_id = "1BRS" # barnase_barstar_complex
complex_ch = "A,D"
out_path = 'data/barnase_barstar/'
os.makedirs(out_path, exist_ok=True)

## Prepare pdbs

In [ ]:
# Downloading desired PDB files
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
barnase_pdb = f'{out_path}{barnase_id}.pdb'
barstar_pdb = f'{out_path}{barstar_id}.pdb'
complex_pdb = f'{out_path}{complex_id}.pdb'

# Create and launch bb
pdb(output_pdb_path=barnase_pdb, properties=def_dict({'pdb_code': barnase_id}))
pdb(output_pdb_path=barstar_pdb, properties=def_dict({'pdb_code': barstar_id}))
pdb(output_pdb_path=complex_pdb, properties=def_dict({'pdb_code': complex_id}))

In [ ]:
# These are the pdbs we get from RCSB
show_pdbs([barnase_pdb, barstar_pdb, complex_pdb])

In [ ]:
# Filtering specific chains: we need to get rid of repeated chains
from biobb_pdb_tools.pdb_tools.biobb_pdb_selchain import biobb_pdb_selchain

# Create properties dict and inputs/outputs
barnase_pdb_ch = f'{out_path}{barnase_id}_ch.pdb'
barstar_pdb_ch = f'{out_path}{barstar_id}_ch.pdb'
complex_pdb_ch = f'{out_path}{complex_id}_ch.pdb'

# # Create and launch bb
biobb_pdb_selchain(input_file_path  = barnase_pdb,
                   output_file_path = barnase_pdb_ch,
                   properties       = def_dict({'chains': barnase_ch}))

biobb_pdb_selchain(input_file_path  = barstar_pdb,
                   output_file_path = barstar_pdb_ch,
                   properties       = def_dict({'chains': barstar_ch}))

biobb_pdb_selchain(input_file_path  = complex_pdb,
                   output_file_path = complex_pdb_ch,
                   properties       = def_dict({'chains': complex_ch}))

In [ ]:
# On a real case we don't have the reference to know how the proteins bind each other
# What information can use to guide the process?
show_pdbs([barnase_pdb_ch, barstar_pdb_ch, complex_pdb_ch])

## Prepare AIRs

In [ ]:
# Solvent accessibility: 
from biobb_haddock.haddock_restraints.haddock3_accessibility import haddock3_accessibility

# Create properties dict and inputs/outputs
barnase_sasa_out = f'{out_path}{barnase_id}_sasa_out.txt'
barstar_sasa_out = f'{out_path}{barstar_id}_sasa_out.txt'
barnase_sasa_actpass = f'{out_path}{barnase_id}_sasa_actpass.txt'
barstar_sasa_actpass = f'{out_path}{barstar_id}_sasa_actpass.txt'

cutoff = 0.3
# Barnase Chain
haddock3_accessibility(
        input_pdb_path            = barnase_pdb_ch,
        output_accessibility_path = barnase_sasa_out,
        output_actpass_path       = barnase_sasa_actpass,
        properties                = def_dict({'chain': barnase_ch,
                                              'cutoff': cutoff}))
# Barstar Chain
haddock3_accessibility(
        input_pdb_path            = barstar_pdb_ch,
        output_accessibility_path = barstar_sasa_out,
        output_actpass_path       = barstar_sasa_actpass,
        properties                = def_dict({'chain': barstar_ch,
                                              'cutoff': cutoff}))

In [ ]:
# Careful! Pockets are good places to bind but have low accessibility
view1 = display_actpass(barnase_pdb_ch, barnase_sasa_actpass)
view2 = display_actpass(barstar_pdb_ch, barstar_sasa_actpass)
ipywidgets.HBox([view1, view2])

In [ ]:
# Electrostatic energies:
# We see a postive charge in the binding site of barnase and a negative charge in the binding site of barstar
show_pdbs([barnase_pdb_ch, barstar_pdb_ch],surface=True)

In [ ]:
# Obtain passive from active selection
from biobb_haddock.haddock_restraints.haddock3_passive_from_active import haddock3_passive_from_active

barnase_pass2act = f'{out_path}{barnase_id}_manual_actpass.txt'
barstar_pass2act = f'{out_path}{barstar_id}_manual_actpass.txt'

haddock3_passive_from_active( 
    input_pdb_path      = barnase_pdb_ch,
    output_actpass_path = barnase_pass2act,
    properties          = def_dict({'active_list' : '27,73,83,87'})
)

haddock3_passive_from_active( 
    input_pdb_path      = barstar_pdb_ch,
    output_actpass_path = barstar_pass2act,
    properties          = def_dict({'active_list' : '33,35,39,43'})
)

In [ ]:
view1 = display_actpass(barnase_pdb_ch, barnase_pass2act, opacity=0.3)
view2 = display_actpass(barstar_pdb_ch, barstar_pass2act, opacity=0.3)
ipywidgets.HBox([view1, view2])

In [ ]:
# Convert active/passive to ambiguous restraints
from biobb_haddock.haddock_restraints.haddock3_actpass_to_ambig import haddock3_actpass_to_ambig

# With SASA
barnase_barstar_sasa_tbl = f'{out_path}barnase_barstar_sasa.tbl'
haddock3_actpass_to_ambig( 
    input_actpass1_path = barnase_sasa_actpass,
    input_actpass2_path = barstar_sasa_actpass,    
    output_tbl_path     = barnase_barstar_sasa_tbl,
    properties          = def_dict({'pass_to_act' : True,  # tbl need actives, we use the passive as active
                                    'segid_one': barnase_ch, 
                                    'segid_two': barstar_ch}))

# With manual active/passive
barnase_barstar_manual_tbl = f'{out_path}barnase_barstar_manual.tbl'
haddock3_actpass_to_ambig( 
    input_actpass1_path = barnase_pass2act,
    input_actpass2_path = barstar_pass2act,    
    output_tbl_path     = barnase_barstar_manual_tbl,
    properties          = def_dict({'segid_one': barnase_ch,
                                    'segid_two': barstar_ch}))

# The restrain have the next format:
# assign (selection1) (selection2) distance, lower-bound correction, upper-bound correction

In [ ]:
# Validate tbl
!haddock3-restraints validate_tbl {barnase_barstar_sasa_tbl} --silent
!haddock3-restraints validate_tbl {barnase_barstar_manual_tbl} --silent

## Docking

In [ ]:
barnase_pdb_ch = f'{out_path}{barnase_id}_ch.pdb'
barstar_pdb_ch = f'{out_path}{barstar_id}_ch.pdb'
complex_pdb_ch = f'{out_path}{complex_id}_ch.pdb'
barnase_barstar_manual_tbl = 'data/barnase_barstar/barnase_barstar_manual.tbl'

### 0. Topology

In [ ]:
from biobb_haddock.haddock.topology import topology

properties=def_dict({
    'cfg': {
        'tolerance': 0,
    },
})

step_idx = 0
barnase_top_zip_path = f'{out_path}{step_idx}/barnase_top.zip'
barstar_top_zip_path = f'{out_path}{step_idx}/barstar_top.zip'
wf_topology          = f'{out_path}{step_idx}/wf_topology.zip'

topology(mol1_input_pdb_path        = barnase_pdb_ch,
         mol2_input_pdb_path        = barstar_pdb_ch,
         mol1_output_top_zip_path   = barnase_top_zip_path,
         mol2_output_top_zip_path   = barstar_top_zip_path,
         output_haddock_wf_data_zip = wf_topology,
         properties                 = properties)

### 1. Rigid body docking

In [ ]:
from biobb_haddock.haddock.rigid_body import rigid_body

properties=def_dict({
    'cfg': {
        'tolerance': 5,
        'sampling': 10,
        # turn on random definiton of AIRs
        'ranair': False
    },
})

step_idx = 1
docking_output_zip_path = f'{out_path}{step_idx}/docking.zip'
wf_rigidbody            = f'{out_path}{step_idx}/wf_rigidbody.zip'

rigid_body(input_haddock_wf_data_zip   = wf_topology,
           docking_output_zip_path     = docking_output_zip_path,
           ambig_restraints_table_path = barnase_barstar_manual_tbl,
           output_haddock_wf_data_zip  = wf_rigidbody,
           properties                  = properties)

In [ ]:
folder = docking_output_zip_path[:-4]
if os.path.exists(folder):
    shutil.rmtree(folder)
if not os.path.exists(folder):
    os.makedirs(folder)

with zipfile.ZipFile(docking_output_zip_path, 'r') as zip_ref:
    zip_ref.extractall(folder)

In [ ]:
import pytraj as pt
import glob

pdb_dir = "data/barnase_barstar/1/docking/"
pdb_files = sorted(glob.glob(f"{pdb_dir}/*.pdb.gz"))
def show_aligned(chain):
    # Get all PDB files and sort them
    # Create a trajectory from the PDB files
    traj = pt.iterload(pdb_files, top=pdb_files[0])
    # Save the trajectory
    # pt.write_traj(f"{pdb_dir}/combined_{chain}_aligned.dcd", traj, overwrite=True)
    pt.align(traj, ref=0, mask=f'::{chain}')
    traj.save(f"{pdb_dir}/combined_{chain}_aligned_clust.pdb", options="model", overwrite=True)
    view = nv.show_pytraj(traj)
    view.layout.width = '100%'
    return view

In [ ]:
view1 = show_aligned('B') # barnase
view2 = show_aligned('A') # barstar

# Display the viewer
ipywidgets.HBox([view1, view2])

In [ ]:
view1 = nv.show_structure_file(f"{pdb_dir}/combined_A_aligned_clust.pdb", default_representation=False)
view2 = nv.show_structure_file(f"{pdb_dir}/combined_B_aligned_clust.pdb", default_representation=False)
view1.add_ribbon(color='chainIndex')
view2.add_ribbon(color='chainIndex')
view1.layout.width = '100%'
view2.layout.width = '100%'
# Display the viewer
box = ipywidgets.HBox([view1, view2])
display(box)
# Create a dropdown widget
opts = ['All']
opts.extend([pdb_file.split('/')[-1].split('.')[0] for pdb_file in pdb_files])
mdsel = ipywidgets.Dropdown(
    options=opts,
    description='Sel. model:',
    disabled=False,
)
display(mdsel)

def on_dropdown_change(change):
    """Handle dropdown selection changes.
    From https://github.com/nglviewer/nglview/issues/765
    """
    if change['type'] == 'change' and change['name'] == 'value': 
        selected_file = change['new']
        if selected_file=='All':
            view1._remote_call('setSelection', target='compList', args=["*"], 
               kwargs=dict(component_index=0))
            view2._remote_call('setSelection', target='compList', args=["*"], 
               kwargs=dict(component_index=0))
        else:
            # Extract model number from the filename
            model_num = selected_file.split('_')[1]
            print(f"Selected model: {model_num}")
            # Update the view with the selected model
            view1._remote_call('setSelection', target='compList', 
                            args=[f"/{model_num}"], 
                            kwargs=dict(component_index=0))
            # You can also update view2 if needed
            view2._remote_call('setSelection', target='compList', 
                            args=[f"/{model_num}"], 
                            kwargs=dict(component_index=0))

# Register the callback function
mdsel.observe(on_dropdown_change, names='value')

### 2. CAPRI eval

In [ ]:
from biobb_haddock.haddock.capri_eval import capri_eval

output_evaluation_zip_path = f'{out_path}2/caprieval.zip'
wf_caprieval               = f'{out_path}2/wf_caprieval.zip'

capri_eval(input_haddock_wf_data_zip  = wf_rigidbody,
           reference_pdb_path         = complex_pdb_ch,
           output_evaluation_zip_path = output_evaluation_zip_path,
           output_haddock_wf_data_zip = wf_caprieval,
           properties = def_dict())

In [ ]:
with zipfile.ZipFile(wf_caprieval, 'r') as zip_ref:
    zip_ref.extractall(wf_caprieval[:-4])
    
#webbrowser.open(f"http://0.0.0.0:8000/{wf_caprieval[:-4]}/analysis/2_caprieval_analysis/report.html")
#!python3 -m http.server

In [ ]:
tsv_dir = wf_caprieval[:-4]+'/2_caprieval/'
# Load the cluster and single data into pandas DataFrames
cluster_df = pd.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pd.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# DockQ: incorrect (<0.23), acceptable (0.23-0.49), medium (0.49-0.80), and high (>=0.80) 
display(single_df.head())
single_df = single_df.sort_values(by='dockq', ascending=False)
display(single_df.head())
display(cluster_df.head())

In [ ]:
import gzip
import shutil
best_pdb = os.path.normpath(os.path.join(tsv_dir, single_df['model'][0]))
# Decompress the .gz file
with gzip.open(best_pdb + '.gz', 'rb') as f_in:
    with open(best_pdb, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
show_pdbs([best_pdb, complex_pdb_ch])

In [ ]:
# the reference and the input proteins have diferent number of residues/atoms, 
# so a fit based on rmsd like pytraj does fails

# # TODO: meter en structure utils
from Bio.PDB import PDBParser, PDBIO
from Bio.PDB.cealign import CEAligner

# Parse the structures
parser = PDBParser(QUIET=True)
structure1 = parser.get_structure("complex_pdb_ch", complex_pdb_ch)
structure2 = parser.get_structure("best_pdb", best_pdb)
    
# Perform CE alignment
aligner = CEAligner()
aligner.set_reference(structure1)
aligner.align(structure2)

# Save structure2 to a PDB file
output_pdb_path = f"{out_path}aligned_structure2.pdb"
io = PDBIO()
io.set_structure(structure2)
io.save(output_pdb_path)

In [ ]:
view = nv.show_structure_file(output_pdb_path)
view.add_component(complex_pdb_ch)
view.clear()
view.component_0.add_cartoon(selection=f':{barnase_ch}', color='red')
view.component_0.add_cartoon(selection=f':{barstar_ch}', color='pink')
view.component_1.clear()
view.component_1.add_cartoon(selection=f':{complex_ch[0]}', color='blue')
view.component_1.add_cartoon(selection=f':{complex_ch[-1]}', color='cyan')
view

### 3. Extend docking

In [ ]:
# Files are relative to the input_haddock_wf_data_zip
cfg ="""
[seletop]
select = 25

[caprieval]
reference_fname = "./data/2_caprieval/1BRS_ch.pdb"

[flexref]
tolerance = 5
ambig_fname = "./data/1_rigidbody/barnase_barstar_manual.tbl"

[caprieval]
reference_fname = "./data/2_caprieval/1BRS_ch.pdb"

[emref]
tolerance = 5
ambig_fname = "./data/1_rigidbody/barnase_barstar_manual.tbl"

[caprieval]
reference_fname = "./data/2_caprieval/1BRS_ch.pdb"
# ====================================================================
"""
haddock_config_path        = f'{out_path}docking-barnase-barstar.cfg'

with open(haddock_config_path, 'w') as config_file:
    config_file.write(cfg)

In [ ]:
from biobb_haddock.haddock.haddock3_extend import haddock3_extend

output_haddock_wf_data_zip = f'{out_path}3/extend_wf.zip'  

haddock3_extend(input_haddock_wf_data_zip  = wf_caprieval,
                haddock_config_path        = haddock_config_path,
                output_haddock_wf_data_zip = output_haddock_wf_data_zip,
                properties = def_dict())

In [ ]:
with zipfile.ZipFile(output_haddock_wf_data_zip, 'r') as zip_ref:
    zip_ref.extractall(output_haddock_wf_data_zip[:-4])
    
webbrowser.open("http://localhost:8888/edit/biobb_wf_haddock/notebooks/data/barnase_barstar/3/extend_wf/analysis/8_caprieval_analysis/report.html")
#!python3 -m http.server

In [ ]:
from IPython.display import IFrame
#display(IFrame(src='/path/to/your/file.html', width=800, height=600))

In [ ]:
tsv_dir = output_haddock_wf_data_zip[:-4]+'/8_caprieval/'
# Load the cluster and single data into pandas DataFrames
cluster_df = pd.read_csv(tsv_dir + 'capri_clt.tsv', sep='\t',comment='#')
single_df = pd.read_csv(tsv_dir + 'capri_ss.tsv', sep='\t',comment='#')

# DockQ: incorrect (<0.23), acceptable (0.23-0.49), medium (0.49-0.80), and high (>=0.80) 
display(single_df.head())
single_df = single_df.sort_values(by='dockq', ascending=False)
display(single_df.head())
display(cluster_df.head())